In [ ]:
# default_exp base

In [ ]:
#export
from speechsep.imports import *

## AudioBase
The current base class for audio which is used for mono and multi-channel audio types.

In [ ]:
#export
def ResampleSignal(sr_new):
    def _inner(sig, sr):
        '''Resample using faster polyphase technique and avoiding FFT computation. Taken from FastaiAudio by LimeAI'''
        if(sr == sr_new): return sig
        sr_gcd = math.gcd(sr, sr_new)
        resampled = resample_poly(sig, int(sr_new/sr_gcd), int(sr/sr_gcd), axis=-1)
        #resampled = resampled.astype(np.float32)
        return resampled
    return _inner

In [ ]:
#export
class AudioBase():
    _show_args={}
    def __init__(self,sig,_sr,fn=None):
        store_attr(self, 'sig,_sr,fn')
        self.data = self.sig
    def __repr__(self): self.listen(); return f'{self.__str__()}'
    def __str__(self): return f'{self.fn}, {self.duration}secs at {self.sr} samples per second'
    def listen(self): display(Audio(self.sig, rate=self.sr))
    @property
    def sr(self): return self._sr
    @sr.setter
    def sr(self, new_sr):
        if self._sr != new_sr: self.sig = ResampleSignal(new_sr)(self.sig, self.sr)
        self._sr = new_sr
    @property
    def duration(self): return len(self.sig)/self.sr

## SpecBase

In [ ]:
#export
class SpecBase():
    _show_args={}
    def __init__(self, data, sr, fn=None):
        store_attr(self, 'data, sr, fn')
        self._plt_params = {}
    @property
    def plt_params(self): return self._plt_params
    @plt_params.setter
    @delegates(plt.pcolormesh)
    def plt_params(self, **kwargs):
        self._plot = partial(plt.pcolormesh, **kwargs)
        self._plt_params = dict(**kwargs)

## MaskBase

In [ ]:
#export
def _check_shape(audio_list):
    lengths = set([audio.shape for audio in audio_list])
    if len(lengths) > 1: ValueError("To generate Masks make sure that the length of files are equal.")
    return lengths.pop()

def join_audios(audio_list, cls=AudioBase):
    if isinstance(audio_list[0], cls): np_list = [audio.data for audio in audio_list]
    mix = np.zeros(_check_shape(np_list), dtype=complex)
    for aud in np_list:
        mix+=aud
    mix /= len(np_list)
    return cls(mix, audio_list[0].sr)

In [ ]:
fn_list = Path("../data/SmallDataset/esc50_sample/").glob(r"*/*")

audio_list = [AudioBase(*load(fn)) for fn in fn_list]
temp = [aud.data for aud in audio_list[1:3]]
s = _check_shape(temp)

In [ ]:
#export
class MaskBase():
    _show_args={}
    def __init__(self, data):
        store_attr(self, 'data')
    @property
    def shape(self):
        return self.data.shape
    @classmethod
    def create(cls, specs, item_cls=AudioBase):
        joined = join_audios(specs, item_cls)
        return [cls.generate(joined, spec) for spec in specs]
    def __mul__(self, spec):
        raise NotImplementedError('This function needs to be implemented before use')
    def __rmul__(self, spec):
        return self*spec
    @classmethod
    def generate(cls, joined, spec):
        raise NotImplementedError('This function needs to be implemented before use')